## 1. pandas를 이용해 csv 파일 읽어오기

In [5]:
# Pandas library import
import pandas as pd

# ​train.csv, test.csv, sample_submission.csv 파일을 각각 train, test, submission 변수에 저장
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

## 2. 전처리
1. train에서는 columns와 'Survived' 컬럼을 사용  
2. test에서는 columns 컬럼만을 사용  
3. 'Age', 'Fare' 컬럼의 결측치를 각 값의 평균으로 대체  
4. train, test 데이터프레임의 Age,Fare 열의 결측값을 Age,Fare 열의 평균 값으로 대체  
5. 'Parch'가 5 이하이고, 'Fare'이 300 이하인 값을 train 데이터에서 제거  
6. apply 함수를 이용하여 female이면 0, 아니라면 1로 값을 할당  
7. 'Embarked' 컬럼에 원-핫 인코딩을 적용  
8. 독립변수(train_x)와 종속변수(train_y)로 분할  
9. 데이터의 불균형을 바로잡기 위해서 SMOTE 알고리즘을 사용  
10. smote에 train_x, train_y를 넣어서 증강된 데이터 생성  
11. 증강된 데이터를 다시 합하여 train_dataset을 확보  

In [6]:
# columns 선택
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train = train[columns + ['Survived']]
test = test[columns]

In [8]:
# Null 처리
mean_age = train['Age'].mean()
mean_fare = train['Fare'].mean()

# 평균값으로 대치
train['Age'] = train['Age'].fillna(mean_age)
test['Age'] = test['Age'].fillna(mean_age)
train['Fare'] = train['Fare'].fillna(mean_fare)
test['Fare'] = test['Fare'].fillna(mean_fare)

In [9]:
# 이상치 제거
train = train[train['Parch'] <= 5]
train = train[train['Fare'] <= 300]

In [10]:
# 범주형 변수 변환 - apply 함수를 이용하여 female이면 0, 아니라면 1로 값을 할당
train['Sex'] = train['Sex'].apply(lambda x: 0 if x == 'female' else 1)
test['Sex'] = test['Sex'].apply(lambda x: 0 if x == 'femael' else 1)

In [11]:
# 원 - 핫 인코딩
train = pd.get_dummies(train, columns=['Embarked'], drop_first=True)
test = pd.get_dummies(test, columns=['Embarked'], drop_first=True)

In [12]:
# 독립변수(train_x)와 종속변수(train_y)로 분할
train_x = train.drop(columns=['Survived'])
train_y = train['Survived']

In [13]:
# 학습 데이터와 검증 데이터로 분할
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y  = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

In [14]:
# 데이터 분균형 교정
from imblearn.over_sampling import SMOTE

# 모델설정
smote = SMOTE(random_state=0)
# train데이터를 넣어 복제함
X_resampled, y_resampled = smote.fit_resample(train_x,list(train_y))

X_resampled['Survived'] = y_resampled
# X_resample에 'Survived'라는 열을 추가하고 그 값을 y_resampled에서 가져와 채움.
train_dataset = X_resampled
# 'Survived' 열이 포함 X_resampled이 train_dataset 변수에 할당. 전처리된 균형 잡힌 훈련 데이터셋임.

## 3. 로지스틱 회귀(Logistic Regression) 모델

sm.Logit.from_formula 에 독립변수들과 train 데이터를 입력  
모델 학습을 진행  
모델에 val_x 데이터를 넣고 값을 예측  
예측 결과 시리즈의 각 원소를 순회하며 원소 x가 0.5 이상인 경우 1을 반환하고, 그렇지 않은 경우 0을 반환  
confusion_matrix, classification_report 함수에 실제값(val_y)과 예측값(y_pred)을 넣고 혼동행렬을 확인  

In [15]:
import statsmodels.api as sm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

formula = """
Survived ~ C(Pclass)+ C(Sex) + scale(Age) + scale(SibSp) + scale(Parch) + scale(Fare) + C(Embarked_Q)+ C(Embarked_S)
"""

model = sm.Logit.from_formula(formula, data=train)
result = model.fit()
y_pred = result.predict(val_x)
y_pred = y_pred.apply(lambda x: 1 if x >= 0.5 else 0)

print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

Optimization terminated successfully.
         Current function value: 0.440644
         Iterations 6
[[83 20]
 [31 44]]
              precision    recall  f1-score   support

           0       0.73      0.81      0.76       103
           1       0.69      0.59      0.63        75

    accuracy                           0.71       178
   macro avg       0.71      0.70      0.70       178
weighted avg       0.71      0.71      0.71       178



## 4. 의사결정나무(Decision Tree) 모델
SMOTE 알고리즘을 적용한 train_dataset의 독립 변수를 나타내는 열들을 선택하고,
종속 변수인 'Survived'를 선택하여 모델을 학습.

In [16]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=6, random_state=0)
model.fit(train_dataset.drop(columns='Survived'),train_dataset['Survived'])

y_pred = model.predict(val_x)
print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

[[89 14]
 [29 46]]
              precision    recall  f1-score   support

           0       0.75      0.86      0.81       103
           1       0.77      0.61      0.68        75

    accuracy                           0.76       178
   macro avg       0.76      0.74      0.74       178
weighted avg       0.76      0.76      0.75       178



**결과해석**  
lr 모델에 비해 dt 모델은 0 클래스는 정밀도, 재현율, f1-scoer이 모두 향상.  
1 클래스는 정밀도, f1-score가 개선. 결과의 전체 정확도도 향상됨.  
두 모델의 차이점은 주로 1과 0클래스의 재현율과 1클래스의 정밀도에 있다.  
dt 모델은 0 클래스를 식별하는 능력과 1 클래스의 정확한 에측에 더 탁월한 성능을 보여준다.  
반면 lr 모델은 1 클래스의 재현율이 상대적으로 낮다.  
전체적으로 dt 모델이 더 좋다.  

## 5. 랜덤 포레스트(RandomForest) 모델


### RandomForest
RandomForestClassifier는 앙상블 학습 기반의 회귀 모델로,
여러 개의 Decision Tree를 사용하여 예측을 수행하는 모델이다.
앙상블 학습은 여러 개의 모델을 조합하여 더 강력하고 안정적인 예측 모델을 만드는 기법이다.

각 데이터셋에 대해 Decision Tree를 독립적으로 구축하고,
이렇게 구축된 다수의 결정 트리를 조합하여 최종 예측 결과를 산출한다.
이 모델은 다중 피처들의 상호작용을 고려하고,
피처의 중요도를 제공하여 변수 선택이나 피처 엔지니어링에 유용하게 사용될 수 있다.
또, 과적합에 강하고 예측 성닝이 뛰어나다는 장점이 있다.

RandomForest 모델을 사용하기 위한 순서는 다음과 같다.
1. RandomForestClassifier 모듈 import
2. RandomForestClassifier 클래스의 인스턴스를 생성하여 model 변수에 할당.
3. model 객체의 fit() 메서드를 호출하여 모델을 학습.

위 과정을 통해 입력 변수와 대상 변수 간의 관계를 학습하여 새로운 입력에 대한 대상 값을 예측할 수 있다.

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=0)
model.fit(train.drop(columns='Survived'),train['Survived'])

y_pred = model.predict(val_x)
print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

[[96  7]
 [33 42]]
              precision    recall  f1-score   support

           0       0.74      0.93      0.83       103
           1       0.86      0.56      0.68        75

    accuracy                           0.78       178
   macro avg       0.80      0.75      0.75       178
weighted avg       0.79      0.78      0.76       178



**결과해석**  
rf 모델의 결과는 dt 모델의 결과에 비해 모드 지표에서 더 좋은 성능을 보여준다.  
0 클래스의 재현율이 0.93로 상승하여, 실제로 0인 샘플 중 93%가 정확하게 예측되었다.  
1 클래스의 정밀도가 0.86으로 향상되었다.
전체 정확도 역시 향상되었다.  
따라서, rf 모델이 더 좋은 성능을 보여주며, dt 모델의 결과를 고도화한 것을 볼 수 있다.  
모델의 성능이 향상되었음을 나타내며, 더 정확한 분류 결과를 보여준다.  
특히, 0 클래스의 재현율이 상승하여 해당 클래스를 높은 정확도로 잘 예측하고 있음을 알 수 있다.  

## 6. XGBoost 모델

### XGBoost
XGBoost는 Extreme Gradient Boosting의 약자이다.  
이 XGBoost라는 모델을 알기 위해서는, 먼저 Boosting에 관해 이해해야 한다.  
​Boosting은 순차적으로 모델의 정확도를 높이는 방법이다.  
Boosting에서는 먼저 전체 학습 데이터에서 일부를 선택한 하위 데이터 세트와 이를 학습할 첫 번째 모델을 만든다.  
그리고 첫 번째 모델이 잘 학습하지 못한 부분을 반영해서 두 번재 데이터 세트와 모델을 만들고, 이런 과정을 반복해서 점진적으로 모델의 정확도를 높인다.  
​이러한 Boosting 기법을 이용하여 구현한 알고리즘은 Gradient Boost가 대표적이다.  
이 Gradient Boost 알고리즘을 병렬 학습이 지원 되도록 구현한 것이 XGBoost이다.  
​Regression, Classification 문제를 모두 지원하며, 성능과 자원 효율이 좋아서 자주 사용되는 알고리즘이다.  

XGBClassifier의 하이퍼 파라미터

- objective: 목적함수
- n_estimators: 트리 수
- tree_method: gpu 사용
- eval_set: 성능 평가를 수행할 데이터 세트
- eval_metric: 조기 종료를 위한 평가 지표
- early_stopping_rounds: 조기 종료 조건, 평가 지표가 향상될 수 있는 반복 횟수
- verbose: 학습 결과 출력 조건 등  

Early Stopping(조기 중단):  
GBM의 경우 n_estimators에 지정된 횟수만큼 학습을 끝까지 수행하지만,   
XGB의 경우 오류가 더 이상 개선되지 않으면 수행을 중단한다.  
만약 n_estimators을 200으로 설정하고, 조기 중단 파라미터값을 50으로 설정하면,  
1부터 200회까지 부스팅을 하다가 50회를 반복하는 동안 학습 오류가 감소하지 않으면 더 이상 부스팅을 진행하지 않고 종료한다.  
조기 중단 기능은 불필요한 학습 시간을 단축시켜 준다는 장점이 있다.  
하지만 이 조기 중단 값을 급격하게 줄이게 되면 모델 성능이 향상될 여디가 있음에도 불구하고 학습이 조기 중단되는 경우 가 발생할 수 있다.

In [18]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state = 0)
model.fit(train.drop(columns='Survived'),train['Survived'])
y_pred = model.predict(val_x)

print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

[[94  9]
 [26 49]]
              precision    recall  f1-score   support

           0       0.78      0.91      0.84       103
           1       0.84      0.65      0.74        75

    accuracy                           0.80       178
   macro avg       0.81      0.78      0.79       178
weighted avg       0.81      0.80      0.80       178



**결과해석**  
rf 모델에 비해서 xgb 모델이 좋은 성능을 보여준다.  
0 클래스는 정밀도, 재현율, f1-score 모두 향상되었다.  
1 클래스는 정밀도와 f1-score가 개선되었다.  
전체 정확도도 향상되었다.  
따라서 xgb 모델이 더 좋은 성능을 보여주며, rf 모델의 결과를 고도화한 것으로 볼 수 있다.  
모델의 성능이 향상되었음을 나타내며, 더 정확한 분류 결과를 보여준다.  
특히, 0 클래스의 재현율이 상승하여 해당 클래스를 높은 정확도로 잘 예측하고 있음을 알 수 있다.  
1 클래스는 정밀도와, f1-score도 개선되었지만, 재현율은 낮아졌다.

## 7. 모델 선택

In [19]:
my_model = XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state = 0)
my_model.fit(train_dataset.drop(columns='Survived'),train_dataset['Survived'])
XGB_pred = my_model.predict(test)

## 8. 제출 준비

In [20]:
y_pred = my_model.predict(test)
submission['Survived'] = y_pred
submission.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0


## 9. submission 파일 저장하기

In [21]:
submission.to_csv('submission.csv', index=False)